# Apresentação ✒️

Notebook destinado à criação e implementação de [Agents](https://www.ibm.com/think/topics/ai-agents), com LangChain, um framework que permite a manipulação de modelos de linguagem (LLM) para a realização de tarefas e personalização de respostas que promovam melhor aderência ao contexto de uso em relação ao modelo utilizado.

Como caso de uso, o objetivo é criar um agent consultor de carreiras que cumpre a tarefa de guiar os estudantes de uma escola fictícia acerca das universidades de seu interesse, bem como outras ações correlacionadas, podendo ser requisitado a elaborar um perfil de um determinado estudante, compará-lo com outro e etc.

Para a sua realização, carreguei os conjuntos de dados necessários e que estarão disponíveis no trecho pertinente, bem como a criação de ferramentas, elementos salutares que permitem à LLM utilizada acessar informações externas, para além de seu domínio prévio de treinamento e realizar ações.

Cada ferramenta foi nomeada, apresentando comentários para facilitar o entendimento de cada trecho ou passo executado. Após a criação das ferramentas, há a elaboração do agent, seguindo as boas práticas de engenharia de prompt, para a definição de comportamento e assertividade da LLM utilizada (em sua formatação me inspirei no ***set*** utilizado pela [CrewAI](https://www.crewai.com)).



## Bibliotecas 📚

In [1]:
!pip install langchain_google_genai google-generativeai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 705.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.6 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv langchain langchainhub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 14.4 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [4]:
import os
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import google.generativeai as genai

from typing import List
from dotenv import load_dotenv

from langchain import hub

from langchain.tools import BaseTool
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.pydantic_v1 import Field, BaseModel
from langchain_core.messages import HumanMessage

from langchain.agents import create_react_agent, create_openai_tools_agent
from langchain.agents import AgentExecutor

from langchain.agents import Tool
from langchain_core.output_parsers import JsonOutputParser


## Definindo a variável de ambiente 🧩

In [5]:
os.environ['GOOGLE_API_KEY'] = "sua-api-key"

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

## Carregando os datasets 💾

Para o estudantes : https://github.com/alura-cursos/3860-langchain-agentes-python/blob/projeto-base/documentos/estudantes.csv

Para as universidades : https://github.com/alura-cursos/3860-langchain-agentes-python/blob/projeto-base/documentos/universidades.csv



In [6]:
estudantes_file = "estudantes.csv"
universidades_file = "universidades.csv"

df_estudantes = pd.read_csv(estudantes_file)
df_universidades = pd.read_csv(universidades_file)

### Visualizando os dataframes 🔎

In [7]:
df_estudantes.head(2)

,NOME,USUARIO,ANO_FORMATURA,SCORE_MATEMATICA,SCORE_PORTUGUES_,SCORE_BIOLOGIA,SCORE_FISICA,SCORE_COMPUTACAO,SCORE_FILOSOFIA,SCORE_PROJETOS,SCORE_ATIVIDADES_SOCIAIS,SCORE_PUBLICACOES,LISTA_AREAS_PREFERIDAS,PROEFICIENCIA_INGLES,PROEFICIENCIA_ESPANHOL,LISTA_PAISES_PREFERIDOS,LISTA_UNIVERSIDADES_PREFERIDAS
0,Ana,ana,2029,4,6,8,8,3,5,5,2,0,['Humanas'],5.5,5.5,"['Brasil', 'Alemanha']","['UNICAMP', 'Technical University of Berlin']"
1,Ariel,ariel,2026,2,7,8,7,2,4,0,8,10,"['Ciências', 'Saúde', 'Artes']",7.5,9.0,"['Austrália', 'Reino Unido']","['University of Sydney', 'Imperial College']"


In [8]:
df_estudantes.shape

(14, 17)

In [9]:
df_universidades.head(2)

,NOME_FACULDADE,PAIS,CRITERIOS_SELECAO,CURSOS_DESTAQUE,PERFIL_DESEJADO
0,USP,Brasil,"Exame de admissão, Notas do Ensino Médio","Medicina, Engenharia, Direito","Inovadores, Comprometidos, Pró-ativos, Pesquis..."
1,UNICAMP,Brasil,"Exame de admissão, Notas do Ensino Médio","Computação, Física, Química","Criativos, Analíticos, Detalhistas"


In [10]:
df_universidades.shape

(18, 5)

## Verificando a qualidade dos dataframes 🔬

Verificando se os dataframes não apresentam dados nulos, NaN ou duplicados

In [11]:
df_estudantes.duplicated().sum()

0

In [12]:
df_universidades.duplicated().sum()

0

In [13]:
df_estudantes.isna().sum().sum()

0

In [14]:
df_universidades.isna().sum().sum()


0

Após a verificação dos dados no dataframe, observou-se que os arquivos estão próprios para uso, uma vez que não apresentam dados NaN, nulos ou duplicados.

# Criando ferramentas com LangChain 🛠️

As ferramentas para a aplicação atual serão realizadas considerando o escopo dos estudantes, seu conjunto de dados, e das universidades presentes.

A partir da união dessas duas ferramentas será possível unir tanto o conhecimento do perfil do usuário e seus pontos fortes e fracos, com as características das faculdades presentes, de modo a prover uma indicação assertiva ao estudante acerca das universidades que ele pode ingressar.

#Ferramenta referente aos estudantes *🎓*

Criando a ferramenta que realiza a extração dos dados dos estudantes no dataframe que armazena a informação desses.

In [76]:
def consulta_estudantes(estudante):

  dados_estudante = df_estudantes[df_estudantes["USUARIO"] == estudante]

  if dados_estudante.empty:
    return {}
  else:
    return dados_estudante.iloc[:1].to_dict()

## Ferramenta que realiza a extração dos dados dos estudantes

In [77]:
# Classe que formata a saída da extração dos estudantes

class ExtratorDeEstudante(BaseModel):

  estudante : str = Field(
      """
      Nome do estudante informado, sempre em letras minúsculas.
      Exemplo : ana, carla, eduardo, bruna, samantha
      """
  )

In [78]:
# Classe que extrai o estudante do dataset,
# servindo como ferramenta ao Agent para a realização
# de sua tarefa.

class DadosEstudantes(BaseTool):

  name = "ExtraiDadosEstudantes"
  description = (
                """
                Esta ferramenta extrai o histórico e preferências de um estudante
                de acordo com o seu histórico.
                """)

  def _run(self, query: str) -> str:

    llm_1 = ChatGoogleGenerativeAI(
        model = "gemini-1.5-pro-latest",
        temperature = 0
        )

    parser = JsonOutputParser(pydantic_model=ExtratorDeEstudante)

    prompt_template_1 = PromptTemplate(
        template = """
                   Extraia o nome do usuário informado no {input} e o retorne
                   num formato JSON. A chave desse formato deve ser 'estudante'
                   e valor deve ser o nome do usuário em letras minúsculas, seguindo
                   o formato de saída especificado.

                   Formato de saída :
                   -------------------------------
                   {formato_saida}
                   -------------------------------
                   """,
        input_variables = ["input"],
        partial_variables={"formato_saida" : parser.get_format_instructions()}
    )

    chain = prompt_template_1 | llm_1 | parser

    response = chain.invoke({"input" : query})

    # Usando método lower para garantir que o nome do estudante
    # esteja no formato mínusculo, necessário para o funcionamento
    # da ferramenta que extrai os dados dos estudantes, a partir do
    # dataset informado.

    estudante =  response["estudante"].lower()

    # Debug prints
    print("\nResponse from LLM:", estudante)

    dados_estudante = consulta_estudantes(estudante)

    return json.dumps(dados_estudante)

## Ferramenta que elabora o perfil dos estudantes

In [79]:
class Nota(BaseModel):

  disciplina : str = Field("Nome da disciplina")
  nota : float = Field("Nota do estudante na disciplina")

class PerfilAcademicoDoEstudante(BaseModel):

  nome : str = Field("Nome do estudante")
  ano_de_conclusao : int = Field("Ano de conclusão do curso.")

  notas : List[Nota] = Field("""
                             Lista de notas das disciplinas
                             e áreas de conhecimento do aluno.
                             """)

  resumo : str = Field("""
                       Identifique as principais características do estudante
                       e a partir delas faça um resumo de seu perfil acadêmico,
                       de modo a torná-lo único.

                       Ainda, informe os pontos fortes do estudante que o torna
                       um ótimo candidato para as instituições de ensino de seu
                       interesse, mas também os pontos fracos, que o tornam um
                       ruim candidato, destacando os pontos a serem melhorados.
                       """)


In [80]:
# Classe que realiza a construção do perfil acadêmico do estudante.

class PerfilAcademico(BaseTool):

  name = "PerfilAcademico"
  description = (
                """
                Essa ferramenta cria um perfil acadêmico do estudante a partir
                de todos os dados obtidos.
                A ferramenta não é capaz de extrair os dados do estudante, sendo necessário
                utilizar, para isso, a ferramenta ExtraiDadosEstudantes.
                """
                )

  def _run(self, query: str) -> str:

    llm_2 = ChatGoogleGenerativeAI(
        model = "gemini-1.5-pro-latest",
        temperature = 0.5
        )

    parser = JsonOutputParser(pydantic_model=PerfilAcademicoDoEstudante)

    prompt_template_2 = PromptTemplate(
        template = """
                   Formate o estudante para o seu perfil acadêmico.
                   Com os dados, identifique as opções de universidades sugeridas
                   e cursos compatíveis com o interesse do aluno.
                   Destaque o perfil do aluno, dando ênfase principalmente naquilo que
                   faz sentido para as instituições de interesse do aluno.

                   Informações do estudante :
                   {dados_estudante}

                   Formato de saída :
                   -------------------------------
                   {formato_de_saida}
                   -------------------------------
                   """,
        input_variables=["dados_estudante"],
        partial_variables={"formato_de_saida" : parser.get_format_instructions()})

    chain = prompt_template_2 | llm_2 | parser

    response = chain.invoke({"dados_estudante" : query})

    return response

# Ferramenta referente às universidades. 🏰

Criando a ferramenta responsável pela interação com o conjunto de dados das universidades.


In [90]:
# Ferramenta que extrai os dados da universidade especificada.

def dados_da_universidade(universidade : str):

  dados_universidade = df_universidades
  dados_universidade['NOME_FACULDADE'] = dados_universidade['NOME_FACULDADE'].str.lower()
  dados_universidade = dados_universidade[dados_universidade["NOME_FACULDADE"] == universidade]

  if dados_universidade.empty:
    return {}
  else:
    return dados_universidade.iloc[:1].to_dict()

# Ferramenta que extrai os dados de todas as universidades.

def dados_das_universidades():

  dados_das_universidades = df_universidades
  return dados_das_universidades.to_dict()


## Ferramenta que realiza a extração de uma universidade.

In [82]:
class ExtratorDeUniversidades(BaseModel):

  universidade : str = Field(
      """
      Nome da universidade informada, sempre em letras minúsculas.
      """
      )

In [83]:
class DadosUniversidades(BaseTool):

  name = "DadosUniversidades"
  description = (
                """
                Esta ferramenta extrai os dados de uma universidade.
                Passe para essa ferramenta como argumento o nome da faculdade.
                """
                )

  def _run(self, query: str) -> str:

    llm_3 = ChatGoogleGenerativeAI(
        model = "gemini-1.5-pro-latest",
        temperature = 0
        )

    parser = JsonOutputParser(pydantic_model=ExtratorDeUniversidades)

    prompt_template_3 = PromptTemplate(
        template = """
                   Extraia o nome da universidade informada no {input} e o retorne
                   num formato JSON. A chave desse formato deve ser 'universidade'
                   e o valor deve ser o nome da universidade em letras minúsculas,
                   seguindo o formato de saída especificado.

                   Formato de saída :
                   -------------------------------
                   {formato_saida}
                   -------------------------------
                   """,
        input_variables = ["input"],
        partial_variables={"formato_saida" : parser.get_format_instructions()}
    )

    chain = prompt_template_3 | llm_3 | parser

    response = chain.invoke({"input" : query})

    # Debug prints
    print("\nResponse from LLM:", response)

    universidade = response["universidade"].lower()

    dados_universidade = dados_da_universidade(universidade)

    return json.dumps(dados_universidade)

## Ferramenta que realiza a extração de todas as universidades

In [84]:
class TodasAsUniversidades(BaseTool):

  name = "TodasAsUniversidades"
  description = (
                """
                Esta ferramenta extrai os dados de todas as universidades.
                Não é necessário nenhum parâmetro de entrada.

                Apenas a use se lhe for requisitada todas as universidades.
                """
                )

  def _run(self, query: str):

    return dados_das_universidades()

## Ferramenta que realiza o perfil acadêmico da universidade

In [85]:
class PerfilAcademicoDasUniversidades(BaseModel):

  name : str = Field("Nome da universidade")

  pais : str = Field("País onde a universidade está localizada.")

  criterio : List[str] = Field("Lista dos critérios de seleção que a universidade possui.")

  cursos : List[str] = Field("Lista de cursos em detaque oferecidos pela universidade.")

  perfil : str = Field("""
                       Identifique o perfil desejado de estudante de acordo com a
                       universidade e faça uma síntese sobre segundo o interesse
                       da instituição.

                       A partir da síntese, estabeleça uma hierarquia de candidatos,
                       como ideal, bom e razoável, reservando para cada um desses o
                       que precisam ter para serem qualificados como tais.
                       """)


In [86]:
class PerfilUniversidade(BaseTool):

  name = "PerfilUniversidade"
  description = (
                """
                Essa ferramenta cria um perfil acadêmico da universidade
                a partir de todos os dados obtidos.

                A ferramenta não é capaz de extrair os dados da(s) universidade(s),
                sendo necessário utilizar, para isso, a ferramenta ExtraiDadosUniversidades.
                """
                )

  def _run(self, query: str) -> str:

    llm_4 = ChatGoogleGenerativeAI(
        model = "gemini-1.5-pro-latest",
        temperature = 0.5
        )

    parser = JsonOutputParser(pydantic_model=PerfilAcademicoDasUniversidades)

    prompt_template_4 = PromptTemplate(
        template = """
                   Elabore um perfil acadêmico para a universidade a partir das
                   informações da(s) universidade(s) em {dados_universidade}.

                   Junto do perfil criado, crie três tipos de candidatos
                   de interesse da universidade, dividindo-os em ideal, bom
                   e razoável, a partir do perfil desejado.

                   Formato de saída :
                   -------------------------------
                   {formato_de_saida}
                   -------------------------------
                   """,
        input_variables=["dados_universidade"],
        partial_variables={"formato_de_saida" : parser.get_format_instructions()}
    )

    chain = prompt_template_4 | llm_4 | parser

    response = chain.invoke({"dados_universidade" : query})

    return response


# Criando o Agent 🤖

In [87]:
class Agent:

  def __init__(self):

    # Instaciando a classe que extrai os dados dos estudantes.
    dados_estudantes = DadosEstudantes()

    # Instanciando a classe que cria o perfil acadêmico do estudante.
    perfil_academico = PerfilAcademico()

    # Instanciando a classe que extrai os dados das universidades.
    dados_universidades = DadosUniversidades()

    # Instanciando a classe que cria o perfil acadêmico das universidades.
    perfil_universidade = PerfilUniversidade()

    # Instanciando a ferramenta que realiza a extração de todas as universidades.
    todas_as_universidades = TodasAsUniversidades()

    # Criando a ferramenta que será utilizada pelo Agent :

    self.tools_dados_estudantes = [
        Tool(
        name = dados_estudantes.name,
        func = dados_estudantes.run,
        description = dados_estudantes.description
        ),
        Tool(
        name = perfil_academico.name,
        func = perfil_academico.run,
        description = perfil_academico.description
        ),
        Tool(
        name = dados_universidades.name,
        func = dados_universidades.run,
        description = dados_universidades.description
        ),
        Tool(
        name = perfil_universidade.name,
        func = perfil_universidade.run,
        description = perfil_universidade.description
        ),
        Tool(
        name = todas_as_universidades.name,
        func = todas_as_universidades.run,
        description = todas_as_universidades.description
        )
        ]

    # Definindo o template a partir do qual a LLM deverá
    # se nortear para a resolução da consulta. Por meio dele,
    # defino a sua role playing, tarefa e saída esperada, aplicando
    # o método conhecido como ReAct (reasoning and act), buscando
    # reproduzir um compotamento de racíocinio com ação, importantes
    # para a produção de agents autônomos.

    prompt = PromptTemplate(
        template="""
            Você é uma experiente consultora de carreiras de uma escola, especializada
            no âmbito da educação. Com base em sua experiência, consegue identificar
            os pontos fortes e fracos do estudantes e, junto de outras características,
            criar um perfil acadêmico para eles.

            Seu objetivo é responder de forma eficiente as perguntas que lhe
            são feitas referentes aos estudantes da escola, sabendo criar um perfil
            para cada aluno com base em suas características, bem como para cada
            instituição de ensino que tem acesso.

            Para responder as consultas, utilize as {tools} informadas,
            se para a resolução de suas tarefas forem necessárias, seguindo
            o formato a seguir :

            ```
            Thought: Do I need to use a tool? Yes
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ```

            Quando você obter a resposta e poder informá-la ao humano,
            ou não precisar a ferramenta para encontrá-la, você DEVE usar
            o seguinte formato:

            ```
            Thought: Do I need to use a tool? [your response here]
            Final Answer: [your response here]
            ```

            A sua tarefa é responder a consulta ```{input}```.

            A resposta esperada deve ser uma resposta rica, detalhada, assertiva
            e informativa, mas sem ser prolixa, atendendo-se ao contexto da
            consulta e ao o que ela pergunta.

            {agent_scratchpad}
            """,
        input_variables=["tools", "input", "tool_names", "agent_scratchpad"]
    )

    model = ChatGoogleGenerativeAI(
      model = "gemini-1.5-pro-latest",
      temperature = 0.5
      )

    self.agent = create_react_agent(
      llm = model,
      tools = self.tools_dados_estudantes,
      prompt = prompt)



In [88]:
# Instanciando o agent.
agent = Agent()

# Executando o agent.

agent_executor = AgentExecutor(
    agent = agent.agent,
    tools = agent.tools_dados_estudantes,
    verbose = True
    )

In [93]:
query = ("""
        Verifique se o perfil da estudante Brenda é aderente para
        as instituições de seu interesse.
        """)

agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...
```tool_code
Thought: Do I need to use a tool? Yes
Action: ExtraiDadosEstudantes
Action Input: BrendaResponse from LLM: brenda
{"NOME": {"7": "Brenda"}, "USUARIO": {"7": "brenda"}, "ANO_FORMATURA": {"7": 2026}, "SCORE_MATEMATICA": {"7": 4}, "SCORE_PORTUGUES_": {"7": 7}, "SCORE_BIOLOGIA": {"7": 1}, "SCORE_FISICA": {"7": 2}, "SCORE_COMPUTACAO": {"7": 1}, "SCORE_FILOSOFIA": {"7": 5}, "SCORE_PROJETOS": {"7": 9}, "SCORE_ATIVIDADES_SOCIAIS": {"7": 0}, "SCORE_PUBLICACOES": {"7": 9}, "LISTA_AREAS_PREFERIDAS": {"7": "['Ci\u00eancias']"}, "PROEFICIENCIA_INGLES": {"7": 6.0}, "PROEFICIENCIA_ESPANHOL": {"7": 7.0}, "LISTA_PAISES_PREFERIDOS": {"7": "['Canad\u00e1', 'Austr\u00e1lia']"}, "LISTA_UNIVERSIDADES_PREFERIDAS": {"7": "['University of Toronto', 'University of Sydney']"}}Thought: Do I need to use a tool? Yes
Action: PerfilAcademico
Action Input: {"NOME": {"7": "Brenda"}, "USUARIO": {"7": "brenda"}, "ANO_FORMATURA": {"7": 2026}, "SCORE_MATEMATICA": {"7": 4

{'input': '\n        Verifique se o perfil da estudante Brenda é aderente para \n        as instituições de seu interesse. \n        ',
 'output': 'Brenda demonstra grande aptidão para projetos e publicações, o que a destaca em ambos os perfis universitários. Suas habilidades se encaixam na busca por candidatos inovadores e engajados da University of Sydney. No entanto, seu interesse em Ciências, com baixo desempenho em áreas como Biologia, Física e Computação, exige cuidado na escolha do curso.  Explorar áreas dentro de Ciências que se alinhem com projetos e publicações é essencial, assim como fortalecer sua base em áreas científicas para a University of Toronto, que exige excelência acadêmica em áreas específicas.'}